<a href="https://colab.research.google.com/github/unicamp-dl/IA025_2022S1/blob/main/ex09/Pedro_Luis_Azevedo_Costa/Aula_9_Exerc%C3%ADcio_175857.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
nome = "Pedro Luís Azevedo Costa"
print(f'Meu nome é {nome}')

Meu nome é Pedro Luís Azevedo Costa


#  Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da Aula 8, mas iremos agora treinar uma rede neural com **duas camadas** de auto-atenção **causais** para prever a próxima palavra de um texto, data as palavras anteriores como entrada. 

Iremos também trabalhar com sequencias de tamanho variável.

Na camada de auto-atenção, não se esqueça de implementar:
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Conexões residuais
- Camada de feed forward (2-layer MLP)


O dataset usado neste exercício (BrWaC) possui um tamanho razoável e você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [2]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Importação dos pacotes

In [3]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook


In [4]:
# Check which GPU we are using
!nvidia-smi

Wed Jun  1 00:20:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [6]:
from typing import List

from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")
dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

def tokenize(text: str, tokenizer):
    # Recomenda-se usar o tokenizer.batch_encode_plus pois é mais rápido.
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


class MyDataset():
    def __init__(self, texts: List[str], tokenizer, max_seq_length: int):
        # Escreva aqui seu código.
        # self.max_seq_length = max_seq_length
        # tokenized_texts = tokenize(texts, tokenizer)
        # self.length = len(tokenized_texts)
        # for tokenized_text in tokenized_texts:
        #   complete_zeros = (max_seq_length - len(tokenized_text)) * [0]
        #   tokenized_phrase = torch.Tensor([]tokenized_text + complete_zeros)
        #   print(tokenized_phrase)

        self.examples = []
        for text in tqdm_notebook(texts):
            token_ids = tokenize(text=[text], tokenizer=tokenizer)[0]
            input_ids = torch.zeros(max_seq_length).type(torch.LongTensor)
            target_ids = torch.zeros(max_seq_length).type(torch.LongTensor)
            # Adiciona SOS
            input_ids[0] = 101
            for i in range(0, min(max_seq_length , len(token_ids) )):
                if i < (len(input_ids) - 1):
                    input_ids[i + 1] = token_ids[i]
                #if i < (len(token_ids)):
                target_ids[i] = token_ids[i]


            #input_ids = token_ids[0: max_seq_length]
            #target_ids = token_ids[1: max_seq_length] + [0]
            #print(input_ids)
            #print(target_ids)
            self.examples.append((input_ids, target_ids)) 

    
        

    def __len__(self):
        # Escreva aqui seu código.
        return len(self.examples)

    def __getitem__(self, idx):
        input_ids, target_id = self.examples[idx]
        return input_ids, target_id


string = ["Ola tudo bem"]
tokenize(string, tokenizer)
dummy_dataset = MyDataset(texts=string, tokenizer=tokenizer, max_seq_length=9)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
print(len(dummy_dataset))




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1 [00:00<?, ?it/s]

1


## Testando se a implementação do MyDataset está correta

In [7]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, max_seq_length=9)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 2
print('Passou no assert de tamanho do dataset.')

first_batch_input, first_batch_target = next(iter(dummy_loader))

print(first_batch_input)
print(first_batch_target)

correct_first_batch_input = torch.LongTensor(
    [[  101,  3396, 10303,   125, 13239,     0,     0,     0,     0],
     [  101,  1660,  5971,   785,   125,  1847, 13779, 15616,     0]])

correct_first_batch_target = torch.LongTensor(
    [[ 3396, 10303,   125, 13239,     0,     0,     0,     0,     0],
     [ 1660,  5971,   785,   125,  1847, 13779, 15616,     0,     0]])
assert torch.equal(first_batch_input, correct_first_batch_input)
assert torch.equal(first_batch_target, correct_first_batch_target)

print('Passou no assert de dataset.')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/2 [00:00<?, ?it/s]

Passou no assert de tamanho do dataset.
tensor([[  101,  3396, 10303,   125, 13239,     0,     0,     0,     0],
        [  101,  1660,  5971,   785,   125,  1847, 13779, 15616,     0]])
tensor([[ 3396, 10303,   125, 13239,     0,     0,     0,     0,     0],
        [ 1660,  5971,   785,   125,  1847, 13779, 15616,     0,     0]])
Passou no assert de dataset.


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [8]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt

File ‘sample-1gb.txt’ already there; not retrieving.



In [9]:
# Load datasets
max_seq_length = 9

train_examples = 500
valid_examples = 100
test_examples = 100

texts = open('sample-1gb.txt').readlines()

print(f'Read {len(texts)} lines.')

max_lines = train_examples + valid_examples + test_examples
#print(f'Truncating to {max_lines} lines.')
#texts = texts[:max_lines]  

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)

Read 250000 lines.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/249800 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 249800
valid examples: 100
test examples: 100


In [12]:
from torch.nn import Linear, Dropout, Embedding, LayerNorm, Sequential, ReLU
from torch import matmul, transpose
from torch.nn.functional import softmax


class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size: int, max_seq_length: int, dim: int, n_layers: int, pad_token_id: int):
        """
        Implements the Self-attention, decoder-only."

        Args:
            vocab_size (int): Size of the input vocabulary.
            max_seq_length (int): Size of the sequence to consider as context for prediction.
            dim (int): Dimension of the embedding layer for each word in the context.
            n_layers (int): number of self-attention layers.
            pad_token_id (int): id of the pad token that will be ignored in the attention.
        """
        # Escreva seu código aqui.
        super(LanguageModel, self).__init__()
        self.vocab_size = vocab_size
        self.max_seq_length = max_seq_length
        self.dim = dim
        self.n_layers = n_layers
        self.pad_token_id = pad_token_id

        self.embedding = Embedding(vocab_size, dim)

        self.W_q = []
        self.W_k = []
        self.W_v = []
        self.W_0 = []

        for i in range(n_layers):
            self.W_q.append(Linear(dim, dim, bias=False))
            self.W_k.append(Linear(dim, dim, bias=False))
            self.W_v.append(Linear(dim, dim, bias=False))
            self.W_0.append(Linear(dim, dim, bias=False))

        self.layer_norm = LayerNorm([max_seq_length, dim])

        dense_input_dim = max_seq_length * dim
        hidden1 = dense_input_dim * 1

        self.dense1 = Linear(dense_input_dim, hidden1)
        self.relu = ReLU()
        self.dropout = Dropout(p=0.2)
        self.dense2 = Linear(hidden1, vocab_size * max_seq_length)


    def self_attention(self, emb_input, exec_number):
        #emb = self.embedding(inputs) # (B, L, D)
        res_layer = emb_input.to(device)
        K = emb_input.to(device) # (B, L, D)
        Q = emb_input.to(device) # (B, L, D)
        V = emb_input.to(device) # (B, L, D)

        K = self.W_k[exec_number].to(device)(K) # (B, L, D) * (D, D) = (B, L, D)
        Q = self.W_q[exec_number].to(device)(Q) # (B, L, D) * (D, D) = (B, L, D)
        V = self.W_v[exec_number].to(device)(V) # (B, L, D) * (D, D) = (B, L, D)

        scores = matmul(Q, K.transpose(2,1)) # (B, L, D) * (B, D, L) = (B, L, L)
        pyramized_scores = scores.tril()
        probs = softmax(pyramized_scores, dim=-1) # (B, L, L)
        E = matmul(probs, V) # (B, L, L) * (B, L, D) = (B, L, D)
        E = self.W_0[exec_number].to(device)(E)
        E_res = E + res_layer
        return E_res #(B, L, D)


        

    def forward(self, inputs):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, max_seq_length)
            
        Returns:
            logits of shape (batch_size, vocab_size)

        
        B = Batch size
        V = Tamanho do vocabulário
        L = Tamanho máximo da sequência
        D = Dimensão de todos os vetores (embeddings)

        """
        # Escreva seu código aqui.

        # inputs - (B, L)
        out = inputs.to(device)
        out = self.embedding(out) # (B, L, D)
        emb_shape = out.shape

        for i in range(self.n_layers):
            out = self.self_attention(out, i)
            out = self.layer_norm(out)

        out = out.view(-1, out.shape[1] * out.shape[2])
        out = self.dense1(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.dense2(out)
        out = out.reshape(emb_shape[0], emb_shape[1], self.vocab_size)

        return out

model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=64,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)

sample_input, _ = next(iter(DataLoader(training_dataset)))
sample_input = sample_input.to(device)
sample_output = model(sample_input)
print(f'sample_input.shape: {sample_input.shape}')
print(f'sample_output.shape: {sample_output.shape}')

sample_input.shape: torch.Size([1, 9])
sample_output.shape: torch.Size([1, 9, 29794])


## Teste o modelo com um exemplo

In [13]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=64,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)

sample_input, _ = next(iter(DataLoader(training_dataset)))
sample_input = sample_input.to(device)
sample_output = model(sample_input)
print(f'sample_input.shape: {sample_input.shape}')
print(f'sample_output.shape: {sample_output.shape}')

sample_input.shape: torch.Size([1, 9])
sample_output.shape: torch.Size([1, 9, 29794])


In [14]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 156960562


## Assert da Perplexidade


In [15]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target, ignore_token_id: int):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, seq_len, vocab_size)
        target: a LongTensor of shape (batch_size, seq_len)

    Returns:
        A float corresponding to the perplexity
    """

    logits = logits.reshape(-1, logits.shape[-1])
    target = target.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target, reduction='mean', ignore_index=ignore_token_id)
    return torch.exp(loss)


n_examples = 1000

train_input_ids, train_target_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
train_input_ids = train_input_ids.to(device)
train_target_ids = train_target_ids.to(device)

logits = model(train_input_ids)


my_perplexity = perplexity(logits=logits, target=train_target_ids, ignore_token_id=tokenizer.pad_token_id)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=7000)
print('Passou o no assert da perplexidade')

my perplexity:              30737
correct initial perplexity: 29794
Passou o no assert da perplexidade


## Laço de Treinamento e Validação

In [16]:
max_examples = 150_000_000
eval_every_steps = 10000
lr = 3e-4


model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=64,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=64, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=64)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

try:
    PATH = "transformer2layers.pt"
    checkpoint = torch.load(PATH)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    n_examples = checkpoint['n_examples']
    train_losses = checkpoint["train_losses"]
except:
    epoch = 0
    loss = 0
    n_examples=0
    train_losses = []


def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input_ids, target_ids):
    model.eval()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    return loss.item()


#train_losses = []
#n_examples = 0
step = epoch
while n_examples < max_examples:
    for train_input_ids, train_target_ids in train_loader:
        loss = train_step(train_input_ids.to(device), train_target_ids.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            EPOCH = step
            PATH = "transformer2layers.pt"
            LOSS = loss

            torch.save({
                        'epoch': step,
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'loss': LOSS,
                        'n_examples':n_examples,
                        'train_losses': train_losses
                        }, PATH)


            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(val_input_ids.to(device), val_target_ids.to(device))
                    for val_input_ids, val_target_ids in validation_loader]))

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(train_input_ids)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 1.24, valid ppl: 18.08
10000 steps; 640000 examples so far; train ppl: 1.17, valid ppl: 14.77
20000 steps; 1280000 examples so far; train ppl: 1.15, valid ppl: 17.43
30000 steps; 1920000 examples so far; train ppl: 1.13, valid ppl: 18.97
40000 steps; 2560000 examples so far; train ppl: 1.13, valid ppl: 21.13
50000 steps; 3200000 examples so far; train ppl: 1.12, valid ppl: 21.65
60000 steps; 3840000 examples so far; train ppl: 1.11, valid ppl: 23.25


KeyboardInterrupt: ignored

## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [17]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(test_input_ids.to(device), test_target_ids.to(device))
        for test_input_ids, test_target_ids in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 27.905463486721786


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [20]:
prompt = 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Eu gosto de comer pizza pois me faz o
Eu gosto de comer pizza pois me faz o que
Eu gosto de comer pizza pois me faz o que a
Eu gosto de comer pizza pois me faz o que a [UNK]
Eu gosto de comer pizza pois me faz o que a [UNK],
Eu gosto de comer pizza pois me faz o que a [UNK], a
Eu gosto de comer pizza pois me faz o que a [UNK], a "
Eu gosto de comer pizza pois me faz o que a [UNK], a " [UNK]
Eu gosto de comer pizza pois me faz o que a [UNK], a " [UNK] a
Eu gosto de comer pizza pois me faz o que a [UNK], a " [UNK] a "
Eu gosto de comer pizza pois me faz o que a [UNK], a " [UNK] a " [UNK]
Eu gosto de comer pizza pois me faz o que a [UNK], a " [UNK] a " [UNK] "
Eu gosto de comer pizza pois me faz o que a [UNK], a " [UNK] a " [UNK] " a
Eu gosto de comer pizza pois me faz o que a [UNK], a " [UNK] a " [UNK] " a "
Eu gosto de comer pizza pois me faz o que a [UNK], a " [UNK] a " [UNK] " a " "
Eu gosto de comer pizza pois me faz o que a [UNK], a " [UNK] a " [UNK] " a " " a
Eu gosto de comer pizz

In [21]:
prompt = 'Posso não concordar com o que você diz, mas defenderei até  '
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Posso não concordar com o que você diz, mas defenderei até a
Posso não concordar com o que você diz, mas defenderei até a nova
Posso não concordar com o que você diz, mas defenderei até a nova um
Posso não concordar com o que você diz, mas defenderei até a nova um novo
Posso não concordar com o que você diz, mas defenderei até a nova um novo a
Posso não concordar com o que você diz, mas defenderei até a nova um novo a um
Posso não concordar com o que você diz, mas defenderei até a nova um novo a um dia
Posso não concordar com o que você diz, mas defenderei até a nova um novo a um dia a
Posso não concordar com o que você diz, mas defenderei até a nova um novo a um dia a que
Posso não concordar com o que você diz, mas defenderei até a nova um novo a um dia a que um
Posso não concordar com o que você diz, mas defenderei até a nova um novo a um dia a que um [UNK]
Posso não concordar com o que você diz, mas defenderei até a nova um novo a um dia a que um [UNK] a
Posso não concordar com o qu

In [22]:
prompt = 'A vantagem de ter péssima memória é divertir-se muitas vezes com  '
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

A vantagem de ter péssima memória é divertir - se muitas vezes com o
A vantagem de ter péssima memória é divertir - se muitas vezes com o novo
A vantagem de ter péssima memória é divertir - se muitas vezes com o novo que
A vantagem de ter péssima memória é divertir - se muitas vezes com o novo que o
A vantagem de ter péssima memória é divertir - se muitas vezes com o novo que o o
A vantagem de ter péssima memória é divertir - se muitas vezes com o novo que o o que
A vantagem de ter péssima memória é divertir - se muitas vezes com o novo que o o que o
A vantagem de ter péssima memória é divertir - se muitas vezes com o novo que o o que o [UNK]
A vantagem de ter péssima memória é divertir - se muitas vezes com o novo que o o que o [UNK],
A vantagem de ter péssima memória é divertir - se muitas vezes com o novo que o o que o [UNK], o
A vantagem de ter péssima memória é divertir - se muitas vezes com o novo que o o que o [UNK], o que
A vantagem de ter péssima memória é divertir - se muitas

## Bonus 1
Quem conseguir a menor perplexidade no dataset de testes ganha 0.5 ponto na média final.

## Bonus 2
Qual é a complexidade (em notação O-grande) da função de geração de texto acima?

Quem responder corretamente a pergunta acima e deixar a função com menor complexidade ganha 0.5 ponto na média final.